# Generate Bollinger bands using dedicated service - F#

### Overview
Bollinger bands were created by John Bollinger, they are envelopes and plotted around the moving average. The width of the bands is based on the standard deviaton of the closing prices from a moving average. 

The purpose of Bollinger Bands is to provide a relative definition of high and low prices of a market. By definition, prices are high at the upper band and low at the lower band.

This sample demonstrates how to request and plot from a dedicated data service **on-demand** Bollinger bands.

### Inputs/outputs
Bollinger bands indicator requires instrument's identifier,date time intervals and number of periods as per inputs and returns data points for the following components:
1. *Middle band* = n-period moving average
2. *Upper band* = Middle band + (k * n-period standard deviation)
3. *Lower band* = Middle band - (k * n-period standard deviation)

Where:
* *n: number of periods*
* *k: factor to apply to the standard deviation value, k=2 as default value*

This sample shows how to request and plot a simple Bollinger bands graph for basis technical analysis using an open source library. 

### Services used
This sample uses *gRPC requests* in order to retrieve Bollinger bands data from the hosted service. The queried endpoint in this script are:
* *TickBollingerService*: to directly retrieve bollinger bands data reponse from the server

### Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.TickAnalytics.V1*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*
    * *XPlot.Plotly* as per display package

***

# Run Bollinger bands sample
### Step 1: Install packages

In [1]:
#i "nuget: file://~/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis, 0.*-pre*"
#r "nuget: XPlot.Plotly.Interactive"
#r "nuget: TaskBuilder.fs"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Restore sources file://~/.nuget/packages/ https://api.nuget.org/v3/index.json Installed Packages Systemathics.Apis, 0.13.2-pre TaskBuilder.fs, 2.1.0 XPlot.Plotly.Interactive, 4.0.6

Loading extensions from `XPlot.Plotly.Interactive.dll`

Configuring PowerShell Kernel for XPlot.Plotly integration.

Installed support for XPlot.Plotly.

In [2]:
open Systemathics.Apis.Helpers
open Systemathics.Apis.Type.Shared.V1
open Systemathics.Apis.Services.TickAnalytics.V1
open Google.Protobuf.WellKnownTypes
open Google.Type
open Grpc.Core
open Grpc.Net.Client
open XPlot.Plotly
open XPlot.Plotly.Interactive

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [3]:
// Get token as metadata
let headers = TokenHelpers.GetTokenAsMetaData();

// Create communication channel
let channel = ChannelHelpers.GetChannel();

### Step 3: Create and process request
To request *Bollinger bands* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Bollinger bands request parameters

#### 3.1 Instrument selection

In [4]:
let ticker = "AAPL"
let exchange = "BATS"

#### 3.2 Bollinger bands parameters

In [5]:
let field = BollingerPrice.Trade
let length = 20
let deviation = 0.4
let sampling = new Duration ( Seconds = 60L )

#### 3.3 Time period delimitation

In [6]:
// Create time intervals
let yesterday = DateTime.Today.AddDays(-1.)
let start = yesterday.AddDays(-50.)
// Build the bars request date interval (we are using Google date time format : we have to cast the dates)
let dateIntervals = 
  new DateInterval(
    StartDate = new Date ( Year = start.Year, Month = start.Month, Day = start.Day ),
    EndDate = new Date ( Year = yesterday.Year, Month = yesterday.Month, Day = yesterday.Day )
  )

// Build the bars request time interval (we are using Google date time format : we have to cast the dates)
// UTC time zone
let timeInterval = 
  new TimeInterval(
    StartTime = new TimeOfDay ( Hours = 15, Minutes = 00, Seconds = 00 ),
    EndTime = new TimeOfDay ( Hours = 16, Minutes = 00, Seconds = 00 ) 
  )

#### 3.4 Request creation
The following code snippets creates *gRPC client*, process request and returns the request reply:

In [7]:
// Generate constraints based on the previous time selection:
let constraints = new Constraints()
constraints.DateIntervals.Add(dateIntervals)
constraints.TimeIntervals.Add(timeInterval)

The following code snippets create requests for Bollinger bands:

In [8]:
// Generate the tick Bollinger Bands request
let request = 
    new TickBollingerRequest(
        Identifier = new Identifier ( Exchange = exchange, Ticker = ticker), 
        Constraints = constraints,  
        Field = field,
        Length = length, 
        Deviation = deviation,
        Sampling = sampling
    )

// Instantiate the tick Bollinger bands service
let service = new TickBollingerService.TickBollingerServiceClient(channel)

### Step 4: Visualize data

#### 4.1 Retrieve Bollinger bands data

In [9]:
open FSharp.Control.Tasks.V2
open System.Collections.Generic

let iterAsync (t: IAsyncEnumerable<'T>) (f : 'T -> unit) =
    let task = 
        task {
            let e = t.GetAsyncEnumerator()
            let mutable go = true
            try
                let! step = e.MoveNextAsync()
                go <- step
                while go do 
                    f e.Current
                    let! step = e.MoveNextAsync()
                    go <- step
            finally
                e.DisposeAsync().AsTask().Wait()
        }
    task.Wait()



let bollingerBands = new List<TickBollingerResponse>()
let call = service.TickBollinger(request, headers)
let e = call.ResponseStream.ReadAllAsync()
iterAsync e bollingerBands.Add

bollingerBands
|> display

index TimeStamp Value Lower Upper Middle 0 Seconds Nanos 1662476460 493600000 155.91 <null> <null> <null> 1 Seconds Nanos 1662476520 313200000 155.92 <null> <null> <null> 2 Seconds Nanos 1662476580 102900000 155.9 <null> <null> <null> 3 Seconds Nanos 1662476640 44900000 155.88 <null> <null> <null> 4 Seconds Nanos 1662476700 4300000 155.83 <null> <null> <null> 5 Seconds Nanos 1662476761 10400000 155.71 <null> <null> <null> 6 Seconds Nanos 1662476820 801300000 155.97 <null> <null> <null> 7 Seconds Nanos 1662476880 4700000 156.05 <null> <null> <null> 8 Seconds Nanos 1662476940 103700000 156.41 <null> <null> <null> 9 Seconds Nanos 1662477000 23100000 156.25 <null> <null> <null> 10 Seconds Nanos 1662477060 59000000 156.09 <null> <null> <null> 11 Seconds Nanos 1662477120 3700000 155.95 <null> <null> <null> 12 Seconds Nanos 1662477183 148200000 155.89 <null> <null> <null> 13 Seconds Nanos 1662477241 493900000 155.98 <null> <null> <null> 14 Seconds Nanos 1662477302 736600000 155.97 <null> <null> <null> 15 Seconds Nanos 1662477360 7700000 155.86 <null> <null> <null> 16 Seconds Nanos 1662477420 671700000 155.79 <null> <null> <null> 17 Seconds Nanos 1662477480 416800000 155.74 <null> <null> <null> 18 Seconds Nanos 1662477540 4100000 155.68 <null> <null> <null> 19 Seconds Nanos 1662477600 76300000 155.71 155.85443174756 155.99456825244005 155.92450000000002 (340 more)

#### 4.2 Plot Bollinger bands
Display Bollinger bands over the look back period with the package of your choice.<br>*`XPlot.Plotly`* is used as per open source display package.
Plot a simple line graph with the *Bollinger bands* as follows: 

In [10]:
// Plot Bollinger bands
[
    new Scattergl(
        name = "Upper band",
        mode = "lines",
        x = (bollingerBands |> Seq.map (fun l -> l.TimeStamp.ToDateTime())),
        y = (bollingerBands |> Seq.map (fun l -> l.Upper))
    )
    new Scattergl(
        name = "Middle band",
        mode = "lines",
        x = (bollingerBands |> Seq.map (fun l -> l.TimeStamp.ToDateTime())),
        y = (bollingerBands |> Seq.map (fun l -> l.Middle))
    )
    new Scattergl(
        name = "Lower band",
        mode = "lines",
        x = (bollingerBands |> Seq.map (fun l -> l.TimeStamp.ToDateTime())),
        y = (bollingerBands |> Seq.map (fun l -> l.Lower))
    )
    new Scattergl(
        name = "Price",
        mode = "markers",
        x = (bollingerBands |> Seq.map (fun l -> l.TimeStamp.ToDateTime())),
        y = (bollingerBands |> Seq.map (fun l -> l.Value))
    )
]
|> Chart.Plot
|> Chart.WithTitle $"Bollinger bands for {ticker}"
|> Chart.WithXTitle "Time"
|> Chart.WithYTitle "Price"
|> Chart.WithWidth 1500
|> Chart.WithHeight 800
|> display

<!DOCTYPE html>